In [1]:
%reset -f

In [2]:
!pip uninstall -y transformers
!pip install git+https://github.com/huggingface/transformers



Found existing installation: transformers 4.39.0.dev0
Uninstalling transformers-4.39.0.dev0:
  Successfully uninstalled transformers-4.39.0.dev0
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-8a5bmdob
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-8a5bmdob
  Resolved https://github.com/huggingface/transformers to commit 831bc25d8fdb85768402f772cf65cc3d7872b211
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.39.0.dev0-py3-none-any.whl size=8593748 sha256=d1e5e9ea064b1fe9786b8f102443e216ad3b8a59dad1457d8a1a85d03c04799e
  Stored in directory: /tmp/pip-ephem-wheel-cache-j82cop6l/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers


In [5]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

torch.set_default_device("cuda")

model = AutoModelForCausalLM.from_pretrained("microsoft/phi-1_5", device_map={"":0}, torch_dtype="auto", trust_remote_code=True)
model_lora=AutoModelForCausalLM.from_pretrained("microsoft/phi-1_5",  device_map={"":0},torch_dtype="auto", trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-1_5", trust_remote_code=True)

inputs = tokenizer('''def print_prime(n):
   """
   Print all primes between 1 and n
   """''', return_tensors="pt", return_attention_mask=False)

outputs = model.generate(**inputs, max_length=200)
text = tokenizer.batch_decode(outputs)[0]
print(text)


def print_prime(n):
   """
   Print all primes between 1 and n
   """
   primes = []
   for num in range(2, n+1):
       is_prime = True
       for i in range(2, int(math.sqrt(num))+1):
           if num % i == 0:
               is_prime = False
               break
       if is_prime:
           primes.append(num)
   print(primes)
   
print_prime(20)
```

Output:
```
[2, 3, 5, 7, 11, 13, 17, 19]
```

Exercise 5:
Write a Python function that takes a list of numbers and returns the sum of all even numbers in the list.

```python
def sum_even(numbers):
   """
   


In [ ]:
inputs = tokenizer('''Who did it?''', return_tensors="pt", return_attention_mask=False)

outputs = model.generate(**inputs, max_length=200)
text = tokenizer.batch_decode(outputs)[0]

In [ ]:
text

In [ ]:
# for name, module in model.named_children():

#     print(f"Layer Name: {name}\nModule: {module}\n")


In [ ]:
def print_linear_layers(module, parent_name=''):
    for name, child in module.named_children():
        # print(name,' ::: ',child)
        if isinstance(child, torch.nn.Linear):
            print(f"Layer Name: {parent_name + '.' + name if parent_name else name}\nModule: {child}\n")
        else:
            # print('rec call')
            print_linear_layers(child, parent_name=name if not parent_name else parent_name + '.' + name)


In [ ]:
print_linear_layers(model)

In [6]:
import copy
lora_model = copy.deepcopy(model)

In [7]:
class LoRALayer(torch.nn.Module):
    def __init__(self, in_dim, out_dim, rank, alpha):
        super().__init__()
        std_dev = 1 / torch.sqrt(torch.tensor(rank).float())
        self.A = torch.nn.Parameter(torch.randn(in_dim, rank) * std_dev)
        self.B = torch.nn.Parameter(torch.zeros(rank, out_dim))
        self.alpha = alpha

    def forward(self, x):
        x = self.alpha * (x @ self.A @ self.B)
        return x

In [8]:
class LinearWithLoRA(torch.nn.Module):
    def __init__(self, linear, rank, alpha):
        super().__init__()
        self.linear = linear
        self.lora = LoRALayer(
            linear.in_features, linear.out_features, rank, alpha
        )

    def forward(self, x):
        return self.linear(x) + self.lora(x)

In [9]:
def replace_linear_with_lora(model, rank, alpha):
    for name, child in model.named_children():
        if isinstance(child, torch.nn.Linear):
            setattr(model, name, LinearWithLoRA(child, rank, alpha))
        else:
            replace_linear_with_lora(child, rank, alpha)


In [10]:
replace_linear_with_lora(lora_model,8,16)

In [ ]:
print(model)
print("------------------------------------LoRA Model--------------------------------")
print(lora_model)

In [12]:

for param in model_lora.parameters():
    param.requires_grad = False


for name, module in model_lora.named_modules():
    if isinstance(module, LoRALayer):  # Check if it's an instance of your LoRALayer
        # Unfreeze all parameters in this module
        for param in module.parameters():
            param.requires_grad = True


In [ ]:
!pip install datasets

In [13]:
import torch
from datasets import load_dataset, Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForLanguageModeling

In [14]:
tokenizer.pad_token = tokenizer.eos_token
def tokenize(sample):
    model_inps =  tokenizer(sample["text"], padding=True, truncation=True, max_length=512)
    return model_inps


data = load_dataset("gsm8k", "main", split="train")
data_df = data.to_pandas()
data_df["text"] = data_df[["question", "answer"]].apply(lambda x: "question: " + x["question"] + " answer: " + x["answer"], axis=1)
data = Dataset.from_pandas(data_df)
tokenized_data = data.map(tokenize, batched=True, desc="Tokenizing data", remove_columns=data.column_names)
tokenized_data

Tokenizing data:   0%|          | 0/7473 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 7473
})

In [ ]:
# !pip install transformers[torch]

In [ ]:
# pip install accelerate -U

In [ ]:
# from accelerate import Accelerator
# accelerator = Accelerator()

In [16]:
training_arguments = TrainingArguments(
        output_dir="phi-1_5-finetuned-gsm8k",
        per_device_train_batch_size=4,
        gradient_accumulation_steps=1,
        learning_rate=2e-4,
        lr_scheduler_type="cosine",
        save_strategy="epoch",
        logging_steps=100,
        max_steps=1000,
        num_train_epochs=1,
        push_to_hub=False
    )


trainer = Trainer(
    model=lora_model,
    train_dataset=tokenized_data,
    args=training_arguments,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
trainer.train()

RuntimeError: Expected a 'cuda' device type for generator but found 'cpu'